In [1]:
#取得上市資料
#requests可以自動添加header提取網頁內容，避免被網站封鎖
import requests
res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
import pandas as pd
df = pd.read_html(res.text)[0]

#移除權證、期貨
def stock_filter(element):
    stock_list = element
    if 'TW0000' in stock_list:
        return False
    elif 'A' in stock_list or 'B' in stock_list or 'C' in stock_list or 'D' in stock_list or 'E' in stock_list or 'F' in stock_list:
        return False
    elif 'KYG' in stock_list:
        return True
    elif 'TW00' in stock_list:
        return True
    else:
        return False
#除DR
def stock_filter2(element):
    stock_list = element
    if 'DR' in stock_list:
        return False
    else:
        return True
    
def stock_num(data):
    data.split(" ")
    return data[0:4]
def stock_name(data):
    data.split(" ")
    return data[4:]

    
#設定columns名子
df.columns = df.iloc[0]

#設定回表格
df = df.iloc[2:, [0,1,2,3,4]]

df["有價證券代號"] = df["有價證券代號及名稱"].apply(stock_num)
df["有價證券名稱"] = df["有價證券代號及名稱"].apply(stock_name)

#改變columns排序
cols=['有價證券代號','有價證券名稱','國際證券辨識號碼(ISIN Code)','上市日','市場別','產業別']
df=df.loc[:, cols]

#過濾
bool_filter = df["國際證券辨識號碼(ISIN Code)"].apply(stock_filter)
bool_filter2 = df["有價證券名稱"].apply(stock_filter2)
df = df[bool_filter]
df = df[bool_filter2]
df.to_csv("stock_上市.csv", index=False ,encoding="utf-8")

C:\Users\keneau\PycharmProjects\work\venv\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [2]:
#取得上櫃資料，僅替換網址

import requests
res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=4")
import pandas as pd
df = pd.read_html(res.text)[0]

#移除權證、期貨
def stock_filter(element):
    stock_list = element
    if 'TW0000' in stock_list:
        return False
    elif 'A' in stock_list or 'B' in stock_list or 'C' in stock_list or 'D' in stock_list or 'E' in stock_list or 'F' in stock_list:
        return False
    elif 'KYG' in stock_list:
        return True
    elif 'TW00' in stock_list:
        return True
    else:
        return False
#移除DR股
def stock_filter2(element):
    stock_list = element
    if 'DR' in stock_list:
        return False
    else:
        return True    
def stock_num(data):
    data.split(" ")
    return data[0:4]
def stock_name(data):
    data.split(" ")
    return data[4:]

    
#設定columns名子
df.columns = df.iloc[0]
#設定回表格
df = df.iloc[2:, [0,1,2,3,4]]


df["有價證券代號"] = df["有價證券代號及名稱"].apply(stock_num)
df["有價證券名稱"] = df["有價證券代號及名稱"].apply(stock_name)
#改變columns排序
cols=['有價證券代號','有價證券名稱','國際證券辨識號碼(ISIN Code)','上市日','市場別','產業別']
df=df.loc[:, cols]

#過濾
bool_filter = df["國際證券辨識號碼(ISIN Code)"].apply(stock_filter)
bool_filter2 = df["有價證券名稱"].apply(stock_filter2)
df = df[bool_filter]
df = df[bool_filter2]
df.to_csv("stock_上櫃.csv", index=False ,encoding="utf-8")

C:\Users\keneau\PycharmProjects\work\venv\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [3]:
#爬取財報狗需資訊
import requests
import json
import pandas as pd

#使用f12裡的Network找到資料網址https://statementdog.com/api/v1/fundamentals/2330/2001/1/2019/4/cf?qbu=false&_=1557410243403
#crawl_dog(stockid, startyear/Quarter, endyear/Quarter) ex:crawl_dog(2330,'2001/1','2019/4')
def crawl_dog(stock_num, starttime, endtime):
    html = 'https://statementdog.com/api/v1/fundamentals/'+str(stock_num)+'/'+starttime+'/'+endtime+'/cf?qbu=false&_=1557410243403'
    content = requests.get(html)
    text = json.loads(content.text)
    return text

In [4]:
import requests
import json
import pandas as pd
from pandas.core.frame import DataFrame
from pandas.errors import MergeError
import numpy as np

def crawl_profit_margin(stock_num, starttime, endtime):
   
    #計算endtime與starttime中總幾季
    startyear = int(starttime.split("/")[0])
    endyear = int(endtime.split("/")[0])
    startmouth = int(starttime.split("/")[1])
    endmouth = int(endtime.split("/")[1])
    sumQ = (endyear-startyear)*4 + (endmouth+1-startmouth)

    #創建空的dataframe
    data = np.array([np.arange(sumQ)]).T
    df =pd.DataFrame(data,columns=['STKID'],)
    json_data = crawl_dog(stock_num, starttime, endtime)
    
    #labels_num=1公司基本資料，從2開始
    labels_num = 2
    

    #共403個標籤
    while labels_num<403:

        try:   
            data_Quarter = json_data[str(labels_num)]['data']
            data_df=DataFrame(data_Quarter)#lIST轉成DataFrame
            data_Name = json_data[str(labels_num)]['label']
            data_df.rename(columns={0:'STKID',1:data_Name},inplace=True)


            #2001-Q1到2019-Q1 共73季
            if len(data_df) == sumQ:
                df = pd.merge(df,data_df)
                #print(df)
                #print(labels_num,"頁")
            else:
                #print(labels_num,"len != 73")
                pass
            labels_num += 1
        except KeyError:
            #print("找不到",labels_num,"頁")
            labels_num += 1
            pass
        except MergeError:
            #print("合併格式不符",labels_num,"頁")
            labels_num += 1
            pass
        except ValueError:
            #print("格式不符",labels_num,"頁")
            labels_num += 1  
    df["STKID"] =  stock_num
    #取代中文字
    df = df.replace('無',0)
    df = df.replace('虧損',0)
    df = df.replace('負',0)
    df = df.replace('負轉正',0)
    df = df.replace('前期為零',0)
    df.to_csv(str(stock_num)+'.csv', index=False ,encoding="utf-8")

In [5]:
crawl_profit_margin(2330,'2001/1','2019/1')

In [6]:
df = pd.read_csv('2330.csv')
df.tail()

,STKID,TimeQ,營收,毛利,營業利益,稅後淨利,單季EPS,近四季EPS,近四季平均EPS,流動資產,...,銷售費用率,管理費用率,研發費用率,折舊,攤銷,資本支出,營業費用,銷售費用,管理費用,研發費用
68,2330,20181,248078671,124974694,96826946,89787574,3.46,13.32,3.33,929116660,...,0.58,1.96,8.23,70462286,1035591,-71980773,26728394,1448092,4851708,20428594
69,2330,20182,233276811,111588104,84428146,72293375,2.79,13.55,3.39,959042679,...,0.63,2.17,8.53,70351470,1057214,-59696676,26440124,1477977,5070594,19891553
70,2330,20183,260347882,123380843,95245181,89098072,3.44,13.51,3.38,859223705,...,0.61,1.79,8.41,72505194,1104623,-69819583,28128452,1585523,4656730,21886199
71,2330,20184,289770193,138042400,107123251,100005385,3.86,13.54,3.39,951679721,...,0.51,1.96,8.18,74805947,1223977,-113908111,30852310,1476236,5686851,23689223
72,2330,20191,218704469,90352125,64266023,61387310,2.37,12.45,3.11,991324815,...,0.67,1.89,9.34,76192468,1355336,-72432829,26018013,1459973,4140729,20417311


In [7]:
#讀入所有股票代號
df = pd.read_csv('stock_上櫃.csv')
df = df.iloc[:,0]
df = df.tolist()

df2 = pd.read_csv('stock_上市.csv')
df2 = df2.iloc[:,0]
df2 = df2.tolist()

for l in df2:
    df.append(l)
stock_list = df
stock_list

[1240,
 1258,
 1259,
 1264,
 1268,
 1333,
 1336,
 1565,
 1566,
 1569,
 1570,
 1580,
 1584,
 1586,
 1591,
 1593,
 1595,
 1597,
 1599,
 1742,
 1752,
 1777,
 1781,
 1784,
 1785,
 1787,
 1788,
 1795,
 1796,
 1799,
 1813,
 1815,
 2035,
 2061,
 2063,
 2064,
 2065,
 2066,
 2067,
 2070,
 2221,
 2230,
 2233,
 2235,
 2596,
 2640,
 2641,
 2643,
 2718,
 2719,
 2724,
 2726,
 2729,
 2732,
 2734,
 2736,
 2740,
 2745,
 2916,
 2924,
 2926,
 2937,
 3064,
 3066,
 3067,
 3071,
 3073,
 3078,
 3081,
 3083,
 3085,
 3086,
 3088,
 3089,
 3092,
 3093,
 3095,
 3105,
 3114,
 3115,
 3118,
 3122,
 3128,
 3131,
 3141,
 3144,
 3147,
 3152,
 3162,
 3163,
 3169,
 3171,
 3176,
 3178,
 3188,
 3191,
 3202,
 3205,
 3206,
 3207,
 3211,
 3213,
 3217,
 3218,
 3219,
 3221,
 3224,
 3226,
 3227,
 3228,
 3230,
 3232,
 3234,
 3236,
 3252,
 3259,
 3260,
 3264,
 3265,
 3268,
 3272,
 3276,
 3284,
 3285,
 3287,
 3288,
 3289,
 3290,
 3293,
 3294,
 3297,
 3303,
 3306,
 3310,
 3313,
 3317,
 3322,
 3323,
 3324,
 3325,
 3332,
 3339,
 3354,

In [ ]:
#爬取個股資訊，並存成csv
for stocks in stock_list:
    crawl_profit_margin(stocks)

In [ ]:
#將資料夾內csv匯入sql中
for stocks in stock_list:
    to_sql(stocks)